In [1]:
!git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), 560 bytes | 560.00 KiB/s, done.
From https://github.com/mercadolibre/fury_user-navigation-research
   95b929b..ea02e39  experiment-design -> origin/experiment-design
Updating 95b929b..ea02e39
Fast-forward
 core/sigir/search.py | 16 ++++++----------
 1 file changed, 6 insertions(+), 10 deletions(-)


In [2]:
from core.imports import *

In [7]:
from core.sigir import search
from core.sigir import utils
from sklearn.pipeline import make_union, make_pipeline

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from core.sigir.descriptor import UserNavDescriptor, PoolingStrategy, CIDDescriptor
from lightgbm import LGBMClassifier
from core.sigir.model import MultiTaskLanguageModel, RollingAverageMultiTaskLanguageModel
from core.sigir.settings import sigir_data_dir
from core.sigir.grid_search import load_trials, build_runs_df
from core.time_it import timeit
from eval_top_models import load_descriptors, get_checkpoints_metrics, get_run_df

In [4]:
raw_X_train, raw_y_train, raw_X_test, raw_y_test = search.load_data(job_name, censored=False, limit=50_000)

raw_y_train = np.asarray(raw_y_train)
raw_y_test = np.asarray(raw_y_test)

2022-11-23 17:32:23 pid 11950 | starting to iterate, updating every 1 seconds
2022-11-23 17:32:24 pid 11950 | 968, speed: 669.43 it/s, elapsed 1.45 seconds
2022-11-23 17:32:25 pid 11950 | 1561, speed: 637.76 it/s, elapsed 2.45 seconds
2022-11-23 17:32:26 pid 11950 | 2347, speed: 680.69 it/s, elapsed 3.45 seconds
2022-11-23 17:32:27 pid 11950 | 2872, speed: 645.54 it/s, elapsed 4.45 seconds
2022-11-23 17:32:28 pid 11950 | 3638, speed: 667.54 it/s, elapsed 5.45 seconds
2022-11-23 17:32:29 pid 11950 | 4494, speed: 696.75 it/s, elapsed 6.45 seconds
2022-11-23 17:32:30 pid 11950 | 4975, speed: 667.76 it/s, elapsed 7.45 seconds
2022-11-23 17:32:31 pid 11950 | 5732, speed: 678.32 it/s, elapsed 8.45 seconds
2022-11-23 17:32:32 pid 11950 | 6527, speed: 690.65 it/s, elapsed 9.45 seconds
2022-11-23 17:32:33 pid 11950 | print interval is now 5s
2022-11-23 17:32:33 pid 11950 | 6948, speed: 661.28 it/s, elapsed 10.51 seconds
2022-11-23 17:32:38 pid 11950 | 10243, speed: 660.54 it/s, elapsed 15.51 se

In [52]:
# def sample(X, y, pct=0.1):
#     indices = np.nonzero(np.random.RandomState(42).random(len(X)) < pct)[0]
#     return [X[i] for i in indices], [y[i] for i in indices]

# raw_X_train, raw_y_train = sample(raw_X_train, raw_y_train, pct=0.5)
# raw_X_test, raw_y_test = sample(raw_X_test, raw_y_test, pct=0.1)

In [47]:
from core.time_it import timeit
from sklearn.metrics import roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier

def lgbm_factory():
    return LGBMClassifier(
        boosting_type='goss',
        n_estimators=100,
        learning_rate=0.01,
        num_leaves=15,
        silent=False, verbosity=0,
        n_jobs=8, force_col_wise=True
    )

def build_pipe(desc, cid_desc):
    return make_pipeline(
        make_union(
            utils.VectorProjector('candidate.description_vector'),
            utils.VectorProjector('query_vector'),
            desc,
            cid_desc
        ),
        lgbm_factory()
    )


def eval_pipe(pipe, fit=True):
    if fit:
        with timeit('fit'):
            pipe.fit(raw_X_train, raw_y_train)
    
    with timeit('eval'):
        y_train_pred = pipe.predict_proba(raw_X_train)[:, 1]
        roc_train = roc_auc_score(raw_y_train, y_train_pred)
        ap_train = average_precision_score(raw_y_train, y_train_pred)

        y_test_pred = pipe.predict_proba(raw_X_test)[:, 1]
        roc_test = roc_auc_score(raw_y_test, y_test_pred)
        ap_test = average_precision_score(raw_y_test, y_test_pred)
        
        
    return {
        'roc_train': roc_train, 'ap_train': ap_train, 
        'roc_test': roc_test, 'ap_test': ap_test,
        'roc_train/seen': roc_auc_score(raw_y_train[was_seen_tr], y_train_pred[was_seen_tr]),
        'roc_train/not_seen': roc_auc_score(raw_y_train[~was_seen_tr], y_train_pred[~was_seen_tr]),

        'roc_test/seen': roc_auc_score(raw_y_test[was_seen_te], y_test_pred[was_seen_te]),
        'roc_test/not_seen': roc_auc_score(raw_y_test[~was_seen_te], y_test_pred[~was_seen_te]),

        'ap_train/seen': average_precision_score(raw_y_train[was_seen_tr], y_train_pred[was_seen_tr]),
        'ap_train/not_seen': average_precision_score(raw_y_train[~was_seen_tr], y_train_pred[~was_seen_tr]),

        'ap_test/seen': average_precision_score(raw_y_test[was_seen_te], y_test_pred[was_seen_te]),
        'ap_test/not_seen': average_precision_score(raw_y_test[~was_seen_te], y_test_pred[~was_seen_te]),

    }


In [46]:
def get_was_seen_mask(X):
    was_seen = []
    for row in X:
        cand_sku = row['candidate']['product_sku_hash']
        was_seen.append(cand_sku in [e['product_sku_hash'] for e in row['session']])
    return np.asarray(was_seen)

was_seen_tr = get_was_seen_mask(raw_X_train)
was_seen_te = get_was_seen_mask(raw_X_test)

In [8]:
cat2vec_fname = fs.join(sigir_data_dir, 'train/cat_avg_desc_emb.pkl')
gru_runs_df = build_runs_df(load_trials(fs.join(sigir_data_dir, 'trials/gru_smaller_sequences')))
rolling_runs_df = build_runs_df(load_trials(fs.join(sigir_data_dir, 'trials/rolling_smaller_sequences')))

36
48


In [9]:
gru_cid, gru_desc, gru_checkpoint_fname = load_descriptors(
    cat2vec_fname, MultiTaskLanguageModel, gru_runs_df.iloc[0], PoolingStrategy.AVG, device='cuda'
)

2022-11-23 17:35:02 pid 11950 | Starting to make emb dict
2022-11-23 17:35:03 pid 11950 | Time for make emb dict is 0.60 seconds
2022-11-23 17:35:03 pid 11950 | Starting to make emb dict
2022-11-23 17:35:04 pid 11950 | Time for make emb dict is 0.72 seconds
2022-11-23 17:35:04 pid 11950 | Starting to make emb dict
2022-11-23 17:35:04 pid 11950 | Time for make emb dict is 0.59 seconds
2022-11-23 17:35:04 pid 11950 | Starting to make emb dict
2022-11-23 17:35:05 pid 11950 | Time for make emb dict is 0.74 seconds


In [19]:
gru_pipe = build_pipe(gru_desc, gru_cid)
gru_metrics = eval_pipe(gru_pipe)

2022-11-23 17:46:29 pid 11950 | Starting to fit
2022-11-23 17:49:36 pid 11950 | Time for fit is 3.11 minutes
2022-11-23 17:49:36 pid 11950 | Starting to eval
2022-11-23 17:51:51 pid 11950 | Time for eval is 2.26 minutes


In [48]:
gru_metrics = eval_pipe(gru_pipe, fit=False)

2022-11-24 09:55:00 pid 11950 | Starting to eval
2022-11-24 09:57:17 pid 11950 | Time for eval is 2.27 minutes


In [13]:
roll_cid, roll_desc, roll_checkpoint_fname = load_descriptors(
    cat2vec_fname, RollingAverageMultiTaskLanguageModel, rolling_runs_df.iloc[0], PoolingStrategy.LAST, device='cuda'
)


2022-11-23 17:40:30 pid 11950 | Starting to make emb dict
2022-11-23 17:40:31 pid 11950 | Time for make emb dict is 0.54 seconds
2022-11-23 17:40:31 pid 11950 | Starting to make emb dict
2022-11-23 17:40:32 pid 11950 | Time for make emb dict is 0.74 seconds
2022-11-23 17:40:32 pid 11950 | Starting to make emb dict
2022-11-23 17:40:32 pid 11950 | Time for make emb dict is 0.55 seconds
2022-11-23 17:40:32 pid 11950 | Starting to make emb dict
2022-11-23 17:40:33 pid 11950 | Time for make emb dict is 0.74 seconds


In [20]:
roll_pipe = build_pipe(roll_desc, roll_cid)
roll_metrics = eval_pipe(roll_pipe)

2022-11-23 17:51:51 pid 11950 | Starting to fit
2022-11-23 17:54:18 pid 11950 | Time for fit is 2.45 minutes
2022-11-23 17:54:18 pid 11950 | Starting to eval
2022-11-23 17:56:12 pid 11950 | Time for eval is 1.90 minutes


In [49]:
roll_metrics = eval_pipe(roll_pipe, fit=False)

2022-11-24 09:57:17 pid 11950 | Starting to eval
2022-11-24 09:59:11 pid 11950 | Time for eval is 1.91 minutes


In [50]:
gru_metrics

{'roc_train': 0.7207063114075388,
 'ap_train': 0.2800873848449208,
 'roc_test': 0.7208374492900769,
 'ap_test': 0.2847900529165191,
 'roc_train/seen': 0.5909778588080526,
 'roc_train/not_seen': 0.6543759466293227,
 'roc_test/seen': 0.5684351978014365,
 'roc_test/not_seen': 0.6476266528576042,
 'ap_train/seen': 0.4672468971775094,
 'ap_train/not_seen': 0.17656086928091816,
 'ap_test/seen': 0.5084547861693316,
 'ap_test/not_seen': 0.1699120721246824}

In [51]:
roll_metrics

{'roc_train': 0.7005447425629412,
 'ap_train': 0.26306832989294693,
 'roc_test': 0.6987019953041389,
 'ap_test': 0.26732806709891715,
 'roc_train/seen': 0.5701027737703075,
 'roc_train/not_seen': 0.6414813755441462,
 'roc_test/seen': 0.5212661081730214,
 'roc_test/not_seen': 0.636881890111139,
 'ap_train/seen': 0.44997854497327455,
 'ap_train/not_seen': 0.1704961900333002,
 'ap_test/seen': 0.4700568220191653,
 'ap_test/not_seen': 0.17152099688792027}

In [ ]:
0.568435/

In [52]:
pd.DataFrame({'roll': roll_metrics, 'gru': gru_metrics})

,roll,gru
roc_train,0.700545,0.720706
ap_train,0.263068,0.280087
roc_test,0.698702,0.720837
ap_test,0.267328,0.284790
roc_train/seen,0.570103,0.590978
roc_train/not_seen,0.641481,0.654376
roc_test/seen,0.521266,0.568435
roc_test/not_seen,0.636882,0.647627
ap_train/seen,0.449979,0.467247
ap_train/not_seen,0.170496,0.176561


In [54]:
0.171521/0.169912

1.0094696077969774